In [23]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

In [24]:
sg_tsv = './training_detection_vm/train.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})
df_feature_detection

,id,features
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...
...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...


In [25]:
sg_tsv = './training_detection_vm/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
df_label_detection

,id,label
0,1000092795,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...
995,1342780478,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [26]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [27]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:02, 341.11it/s]


In [28]:
full_detection

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [29]:
sg_tsv = './features_detection/training/train.feature.tsv'
df_feature_detection2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_feature_detection2 = df_feature_detection2.rename(columns={0: "id", 1: "features"})
df_feature_detection2

,id,features
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...
...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...


In [30]:
sg_tsv = './features_detection/training/train.label.tsv'
df_label_detection2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_label_detection2 = df_label_detection2.rename(columns={0: "id", 1: "label"})
df_label_detection2

,id,label
0,1000092795,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...
995,1342780478,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [31]:
full_detection2 = pd.merge(df_feature_detection2, df_label_detection2, on="id")
full_detection2

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [32]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection2.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:03, 322.27it/s]


In [34]:
full_detection

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [33]:
full_detection2

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [7]:
# val

In [8]:
sg_tsv = './training_detection_vm/validation.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})
df_feature_detection

,id,features
0,1018148011,{'features': 'AlyjPQAAAAAAAAAAAAAAALdt2T0AAAAA...
1,1029450589,{'features': 'y/T+PsTyTD8AAAAAxmYSPqyrsj4AAAAA...
2,1029737941,{'features': 'a+xjPdZPMUEAAAAASKzVPx/QGUEAAAAA...
3,103205630,{'features': 'WCzwPgAAAAAAAAAAAAAAAHzvrD4AAAAA...
4,10350842,{'features': 'U8KtPgAAAAAAAAAAudo3PubILz8AAAAA...
...,...,...
1009,86120374,{'features': 'ZPrGPQAAAAAAAAAASnvcP5qNZ0EfyqA7...
1010,86131989,{'features': 'mdCSPwAAAAAAAAAA8EE4PbGDmEAAAAAA...
1011,91875542,{'features': 'k/rdPQAAAAAAAAAAAAAAAHzfjD0AAAAA...
1012,981727078,{'features': 'AAAAAAAAAAAAAAAAAAAAAH3AYzz4gis9...


In [17]:
sg_tsv = './training_detection_vm/validation2.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection = df_label_detection.rename(columns={0: "id", 1: "label"})
df_label_detection

,id,label
0,1018148011,"[{'class': 'Sky', 'conf': 0.9436017274856567, ..."
1,1029450589,"[{'class': 'Wall', 'conf': 0.9430888891220093,..."
2,1029737941,"[{'class': 'Headphone', 'conf': 0.966325402259..."
3,103205630,"[{'class': 'Bottle', 'conf': 0.785697400569915..."
4,10350842,"[{'class': 'Sky', 'conf': 0.9426814317703247, ..."
...,...,...
1009,86120374,"[{'class': 'Hair', 'conf': 0.9508698582649231,..."
1010,86131989,"[{'class': 'Shirt', 'conf': 0.8690727949142456..."
1011,91875542,"[{'class': 'Glasses', 'conf': 0.94794499874114..."
1012,981727078,"[{'class': 'Sky', 'conf': 0.9017960429191589, ..."


In [14]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_detection_vm/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'validation2.label.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [15]:
tsv_writer(df_label_detection[["id", "label"]].values.tolist(),LABEL_FILE)

In [18]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,1018148011,{'features': 'AlyjPQAAAAAAAAAAAAAAALdt2T0AAAAA...,"[{'class': 'Sky', 'conf': 0.9436017274856567, ..."
1,1029450589,{'features': 'y/T+PsTyTD8AAAAAxmYSPqyrsj4AAAAA...,"[{'class': 'Wall', 'conf': 0.9430888891220093,..."
2,1029737941,{'features': 'a+xjPdZPMUEAAAAASKzVPx/QGUEAAAAA...,"[{'class': 'Headphone', 'conf': 0.966325402259..."
3,103205630,{'features': 'WCzwPgAAAAAAAAAAAAAAAHzvrD4AAAAA...,"[{'class': 'Bottle', 'conf': 0.785697400569915..."
4,10350842,{'features': 'U8KtPgAAAAAAAAAAudo3PubILz8AAAAA...,"[{'class': 'Sky', 'conf': 0.9426814317703247, ..."
...,...,...,...
1009,86120374,{'features': 'ZPrGPQAAAAAAAAAASnvcP5qNZ0EfyqA7...,"[{'class': 'Hair', 'conf': 0.9508698582649231,..."
1010,86131989,{'features': 'mdCSPwAAAAAAAAAA8EE4PbGDmEAAAAAA...,"[{'class': 'Shirt', 'conf': 0.8690727949142456..."
1011,91875542,{'features': 'k/rdPQAAAAAAAAAAAAAAAHzfjD0AAAAA...,"[{'class': 'Glasses', 'conf': 0.94794499874114..."
1012,981727078,{'features': 'AAAAAAAAAAAAAAAAAAAAAH3AYzz4gis9...,"[{'class': 'Sky', 'conf': 0.9017960429191589, ..."


In [ ]:
# test

In [20]:
sg_tsv = './training_detection_vm/test.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})
df_feature_detection

,id,features
0,1007129816,{'features': 'AAAAAAAAAAAAAAAAAAAAAPXb+T5GtMtA...
1,1009434119,{'features': 'MGCwP7rIGD8AAAAADmf2P/t0MzwAAAAA...
2,101362133,{'features': 'xtUpPdry1j70gqs9uakFP0iUaT8AAAAA...
3,102617084,{'features': 'eiPzPuvL90AAAAAAMWdqPgrM7kAAAAAA...
4,10287332,{'features': 'PYSNPnWZ0z44gPc+AAAAAAAAAAAAAAAA...
...,...,...
995,900144365,{'features': 'zmRlPgAAAAAAAAAAAAAAAA38KUAAAAAA...
996,94024624,{'features': 'D1PgPQAAAAAAAAAAAAAAAC46jUAAAAAA...
997,95758790,{'features': 'Kfx7P5dZ0DwAAAAAmWwaQAAAAAAAAAAA...
998,97233789,{'features': '61RLPyLOaD4z3JY+AAAAAAAAAAAAAAAA...


In [19]:
sg_tsv = './training_detection_vm/test.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection = df_label_detection.rename(columns={0: "id", 1: "label"})
df_label_detection

,id,label
0,1007129816,"[{'class': 'Ear', 'conf': 0.970589816570282, '..."
1,1009434119,"[{'class': 'Dog', 'conf': 0.9937595725059509, ..."
2,101362133,"[{'class': 'Pant', 'conf': 0.8839820623397827,..."
3,102617084,"[{'class': 'Helmet', 'conf': 0.903003513813018..."
4,10287332,"[{'class': 'Man', 'conf': 0.9057866334915161, ..."
...,...,...
995,900144365,"[{'class': 'Short', 'conf': 0.9517521262168884..."
996,94024624,"[{'class': 'Hat', 'conf': 0.9739574790000916, ..."
997,95758790,"[{'class': 'Tree', 'conf': 0.9424041509628296,..."
998,97233789,"[{'class': 'Man', 'conf': 0.9472337365150452, ..."
